## Nonrandom factors and activity coefficients for a five compound mixture

In this notebook we calculate the nonrandom factors for an extremelly nonideal system of five compounds.
This system was first investigated in a very interesting work by
<a href="https://doi.org/10.1016/0378-3812(86)85065-8">Larsen and Rasmussen (1986)</a>.

We then compare activity coefficient and nonrandom factors calculated by SAC equations and the ones originally calculated
based on quasi-chemical theory. We can see the values are in very close agreement.
The very small differences observed are probably due to the numerical convergence tolerance or small differences in the
gas constant assumed.

In [1]:
import math, os, sys
import pandas as pd

parent_directory = os.path.dirname(os.path.abspath('.'))
sys.path.append(parent_directory)

from pysac import SAC, RGAS, RGAS_SI

### Interaction energies

Here we define the interaction energies for the possible pairs, as in the original work.

In [2]:
# Interaction energies in kJ/mol as defined by Larsen and Rasmussen (1986), Table 1.
# https://doi.org/10.1016/0378-3812(86)85065-8
u = [[  0, -20, 20,   5, 30],
     [-20,   0, 10, -30,  2],
     [ 20,  10,  0,  20, 30],
     [  5, -30, 20,   0, 10],
     [ 30,   2, 30,   10, 0]
     ]

# Coordination number assumed in the original work
z = 10

class FiveCompounds(SAC):
    def calc_u(self, T, i, j, m, n):
        '''
        Interaction energies converted to kcal/mol and divided by z to get it per contact
        '''
        return u[i][j] * (RGAS * 1000 / RGAS_SI) / z



### Define the mixture and calculate the activity coefficients

In [3]:
Q_eff = 1

# Five compounds, all with z area
Q = [[z*Q_eff], [z*Q_eff], [z*Q_eff], [z*Q_eff], [z*Q_eff]]

T = 298.15
x = [0.2, 0.2, 0.2, 0.2, 0.2]
# Create an object with the example
pairs = FiveCompounds(Q_eff=Q_eff)
pairs.set_compounds(Q)
pairs.set_temperature(T)
pairs.set_composition(x)
# Force updating all internal variables
lnGamma = pairs.calc_ln_gamma()
gamma = [math.exp(lnGammai) for lnGammai in lnGamma]

# Activity coefficients calculated by Larsen and Rasmussen (Table 3 of their work)
gammaLarsen = [4.2818, 0.0044, 68.67, 0.4091, 45.67]

data = {'Larsen and Rasmussen': gammaLarsen, 'SAC equations (this work)': gamma}
df = pd.DataFrame(data)
df.style.set_caption('''Activity coefficients calculated by Larsen and Rasmussen (1986) 
    using quasi-chemical equations and by our SAC equations.''')

,Larsen and Rasmussen,SAC equations (this work)
0,4.2818,4.28187
1,0.0044,0.00435669
2,68.67,68.6842
3,0.4091,0.409027
4,45.67,45.6775


### Nonrandom factors compared to the original work

We can also calculate the nonrandom factors for this system. This is how much a particular pair is more likely to happen in the mixture when compared to a completelly random solution.

Below we can see that the SAC equation values match the ones reported by <a href="https://doi.org/10.1016/0378-3812(86)85065-8">Larsen and Rasmussen (1986)</a> from quasi-chemical equations:
```
    1.3376, 1.5048, 0.7879, 0.8645, 0.5053
    1.5048, 0.3372, 0.5922, 1.7810, 0.7850
    0.7879, 0.5922, 2.3300, 0.6230, 0.6670
    0.8645, 1.7810, 0.6230, 0.8363, 0.8953
    0.5053, 0.7850, 0.6670, 0.8953, 2.1475
```

In [4]:
alpha = pairs.get_nonrandom()
# put the array in a format we can display
alphaij = [[0 for x in range(len(x))] for y in range(len(x))]
for i in range(len(x)):
    for j in range(len(x)):
        alphaij[i][j] = alpha[i][j][0][0]

df1 = pd.DataFrame(alphaij)
df1.style.set_caption('''Nonrandom factors calculated with SAC equations.''')

,0,1,2,3,4
0,1.3376,1.50475,0.787887,0.864455,0.505308
1,1.50475,0.337156,0.592119,1.78102,0.78496
2,0.787887,0.592119,2.33008,0.622983,0.666928
3,0.864455,1.78102,0.622983,0.836277,0.895268
4,0.505308,0.78496,0.666928,0.895268,2.14754
